
## Input File Requirement to Generate Publications:
#### 1. Name variation and formatting.
#### 2. Exclude society
#### 3. Org(affilition) translation and formatting

## File Format:
##### The First 8 columns of your input file should orderly include: ['External ID', 'First Name', 'Last Name', 'Org - EN', 'MatchTo','Department - EN', 'Input Name','Org_EN']. Any name is ok but you should not change the order.
##### You have to name all the 8 variables even you don't use some.
#####  The only needed variables are first 4. The rest variables will be kept as the input for later comparation.


In [1]:
# import necessary libraries 
# Entrez is one library adapting the E-Utilities functions of Pubmed search for Python
# Note: no more than 3 queries/second using E-Utilities or more than 100,000 results / query -- will require setting up API Key from Pubmed
from Bio import Entrez
import pprint 
import numpy as np
import pandas as pd                 
from datetime import datetime
import os
import re     
import unidecode as un
#Register the API in PubMed!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
Entrez.email = "juan.lin@h1insights.com"
Entrez.api_key = "ba80fd9bb7e3c557cdaca8d5d8314fbfb709"

In [2]:
# function to generate dataframe representing Pubmed search for one person with article information: Name, Organization Affiliation, Title of Paper, DOI 
def generator(PMID, record):
    # initiate dictionaries for storing the different columns of information, will at end convert dictionary to series to pandas dataframe
    # dictionary adds additional security because keeps PMID value for the article information we're seeking
    dicts = {}
    dicts2 = {}
    dicts3 = {}
    dicts4 = {}
    title = []                           #article title - /ArticleTitle 
    CT = []                              #clinical trial - /DataBankList
    date = []                           #publication date - /Journal/PubDate
    peopleID = []                        #
    journal = []                        #journal title - /Journal/Title
    matchto = []
    #Conditional - Do not active!
    #org =[]  #Taiwan
    #inputname = []
    #department =[]
    #inputaffiliation = [] #China



    # iterates through the individual articles
    # length of pubmed articles in records (not PMID, because this will not include book articles (included in records at very end))
    for i in range(len(record['PubmedArticle'])):
        # initialize list for individual article doi, affiliation, name 
        peopleID.append(ID)
        matchto.append(info)
        #Conditional
       # inputname.append(inputname)
        #org.append(org)
        #department.append(department)
        #matchto.append(info)
       # inputaffiliation.append(inputaffiliation) #China
        
        
        doi = []                        #doi - /ELocationID & EIdType == "doi"
        affiliation = []
        mesh = []                       #Mesh -/MeshHeadingList
        name = []                       #Author Name - /AuthorList/ForeName + '' + /AuthorList/LastName
        # Titles of articles in list
        title.append(record['PubmedArticle'][i]['MedlineCitation']['Article']['ArticleTitle'])
        # Journal of which articles were published in 
        journal.append(record['PubmedArticle'][i]['MedlineCitation']['Article']['Journal']['Title'])
        # Existence of clinical trial link in the Article                                                                                    
        if 'DataBankList' in record['PubmedArticle'][i]['MedlineCitation']['Article']:
            CT.append(record['PubmedArticle'][i]['MedlineCitation']['Article']['DataBankList'][0]['AccessionNumberList'][0])
        else: 
            CT.append("NA")
        # Publication date of paper
        if 'PubDate' in record['PubmedArticle'][i]['MedlineCitation']['Article']['Journal']['JournalIssue']:
            if 'Year' in record['PubmedArticle'][i]['MedlineCitation']['Article']['Journal']['JournalIssue']['PubDate']:
                if 'Month' in record['PubmedArticle'][i]['MedlineCitation']['Article']['Journal']['JournalIssue']['PubDate']:
                    year = record['PubmedArticle'][i]['MedlineCitation']['Article']['Journal']['JournalIssue']['PubDate']['Year']
                    month = record['PubmedArticle'][i]['MedlineCitation']['Article']['Journal']['JournalIssue']['PubDate']['Month']
                    fulldate = month + ' ' + year
                    date.append(fulldate)
                else: 
                    date.append(record['PubmedArticle'][i]['MedlineCitation']['Article']['Journal']['JournalIssue']['PubDate']['Year'])     
            else: 
                date.append("NA")                                                                                                           
        for j in range(len(record['PubmedArticle'][i]['MedlineCitation']['Article']['ELocationID'])):
            strings = record['PubmedArticle'][i]['MedlineCitation']['Article']['ELocationID'][j]
            # searches and selects for ID type that is DOI (PII also found here)
            if strings.attributes['EIdType'] == "doi": 
                doi.append(strings)
        # one PMID, one DOI 
        dicts[PMID[i]] = doi
        # iterates through author list to search for correct person and affiliation
        for k in range(len(record['PubmedArticle'][i]['MedlineCitation']['Article']['AuthorList'])):                                    
            authors = record['PubmedArticle'][i]['MedlineCitation']['Article']['AuthorList'][k]
            # if author has an author affiliation
            if authors['AffiliationInfo'] != [] and 'LastName' and 'ForeName' in authors:               
                # match on first and last name 
                    if un.unidecode(authors['LastName']).upper() == un.unidecode(authorlast.upper()) and un.unidecode(authors['ForeName'].upper()) == un.unidecode(authorfirst.upper()):    
                        firstname = authors['ForeName']
                        lastname = authors['LastName']
                        fullname = firstname + ' ' + lastname
                        name.append(fullname)
                    # collect all affiliations attributed to the matched author
                        for l in range(len(authors['AffiliationInfo'])):
                            affiliation.append(authors['AffiliationInfo'][l]['Affiliation'])
            # in the cases for which author affiliation is blank, assign to the affiliation of the first   
            elif 'LastName' and 'ForeName' in authors and un.unidecode(authors['LastName'].upper()) == un.unidecode(authorlast.upper()) and un.unidecode(authors['ForeName'].upper()) == un.unidecode(authorfirst.upper()): 
                if record['PubmedArticle'][i]['MedlineCitation']['Article']['AuthorList'][0]['AffiliationInfo'] != []:
                    firstname = authors['ForeName']
                    lastname = authors['LastName']
                    fullname = firstname + ' ' + lastname
                    name.append(fullname)
                    for l in range(len(record['PubmedArticle'][i]['MedlineCitation']['Article']['AuthorList'][0]['AffiliationInfo'])):
                        conditionalaff = "FIRST AUTHOR AFFILIATION: " + record['PubmedArticle'][i]['MedlineCitation']['Article']['AuthorList'][0]['AffiliationInfo'][l]['Affiliation']
                        affiliation.append(conditionalaff)
             #   elif record['PubmedArticle'][i]['MedlineCitation']['Article']['AuthorList'][-1]['AffiliationInfo'] != []:
            #        firstname = authors['ForeName']
             #       lastname = authors['LastName']
             #       fullname = firstname + ' ' + lastname
             #       name.append(fullname)
             #       for l in range(len(record['PubmedArticle'][i]['MedlineCitation']['Article']['AuthorList'][-1]['AffiliationInfo'])): #NEW
              #          conditionalaff = "LAST AUTHOR AFFILIATION: " + record['PubmedArticle'][i]['MedlineCitation']['Article']['AuthorList'][0]['AffiliationInfo'][l]['Affiliation'] #NEW
              #          affiliation.append(conditionalaff) #NEW
              #          

        # iterate through mesh headings 
        if 'MeshHeadingList' in record['PubmedArticle'][i]['MedlineCitation']:
            for m in range(len(record['PubmedArticle'][i]['MedlineCitation']['MeshHeadingList'])): 
                # [:] allows for getting all of the inside (no string element values) when exporting the dataframe into excel 
                keywords = record['PubmedArticle'][i]['MedlineCitation']['MeshHeadingList'][m]['DescriptorName'][:]
                mesh.append(keywords)
            
        dicts2[PMID[i]] = affiliation
        dicts3[PMID[i]] = name
        dicts4[PMID[i]] = mesh
        
    # dictionary to series to dataframe, named columns
    dicts = pd.Series(dicts).to_frame()                                                                                                                               
    dicts.columns = ['DOI']
    dicts2 = pd.Series(dicts2).to_frame()
    dicts2.columns = ['Affiliation']
    dicts2.Affiliation.apply(tuple)                                                                                                                                    
    dicts3 = pd.Series(dicts3).to_frame()
    dicts3.columns = ['Name']
    dicts4 = pd.Series(dicts4).to_frame()
    dicts4.columns = ['Mesh']    
    
    #join into one dataframe, nice formatting
    publications = dicts.join(dicts2).join(dicts3)
    #publications.DOI = publications.DOI.str[0]
    #publications.Name = publications.Name.str[0]
    publications['Title'] = title
    publications['Clinical Trials'] = CT
    publications['Date'] = date
    publications['PeopleID'] = peopleID
    publications['Journal'] = journal
    publications['Match To'] = matchto
    #Conditional
    publications['Department'] = department 
    #publications['Input Affiliation'] = org   #Taiwan
    publications['Input Affiliation'] = inputaffiliation  #China
    publications['Input Name'] = inputname
    #China
    #publications['Input Affiliation'] = inputname
    
    publications = publications.join(dicts4)
    
    return publications


----
# CASE 1: Search with search terms

Set your directory here:

In [3]:
# Set your directory
os.chdir('C:/Users/Juan/Desktop/ipython')
df = pd.read_excel("test.xlsx", index_col = None)  

In [4]:
# pd.set_option('max_colwidth', 300)
pd.set_option('display.max_rows', 6)
df

,Row,first,last,info,a,b,c,d
0,2,ANTONIO DAVID,LAZARO SANCHEZ,Oncology Spain,Oncology Spain,NaN,NaN,ANTONIO DAVID LAZARO SANCHEZ
1,3,PATRICK,LE COZ,Neurology France,Neurology France,NaN,NaN,PATRICK LE COZ
2,4,Teresa,Leahy,Hospital Nurse|Neurology United Kingdom,Hospital Nurse|Neurology United Kingdom,NaN,NaN,Teresa Leahy
...,...,...,...,...,...,...,...,...
111,116,Petra,Lubinsky,Nurse's Aide Germany,Nurse's Aide Germany,NaN,NaN,Petra Lubinsky
112,117,Michele,Luboja,Nurse's Aide Germany,Nurse's Aide Germany,NaN,NaN,Michele Luboja
113,118,MATTEO,LUCCHINI,Neurology Italy,Neurology Italy,NaN,NaN,MATTEO LUCCHINI


In [5]:
publications=pd.core.frame.DataFrame()
import time
start=time.time()
placeholder = []
noresults1 = []
noresults2 = []
noresults3 = []
for row in df.itertuples(): 
    ID = row[1]
    authorfirst = row[2]
    authorlast = row[3]
    org = row[4]   #Taiwan
    info = row[5]
    #extra
    department = row[6]
    inputname = row[7]
    inputaffiliation = row[8] #China
    
    
    #    #    #    #    #    #    #    #    #    #    #    #    #    #    #    #    ## set search 
    search = '({} {}[Author]) AND ({}[Affiliation])'.format(authorfirst, authorlast, org)
    search = '{} {} {}'.format(authorfirst, authorlast, org)
    # search = '{} {}[Author]]'.format(authorfirst, authorlast)
    #    #    #    #    #    #    #    #    #    #    #    #    #    #    #    #    #    #    #    #    #    #    #    
    
    
    # E-utiltiies esearch searches pubmed, returns max of 500 articles
    handle = Entrez.esearch(db = "pubmed", term = search, retmax = 5)  #set retmax = 5 for Taiwan
    # read parses and returns in simplified format
    record = Entrez.read(handle)
    # in the case that search results exist
               # in the case that search results exist
    if 'ErrorList' in record: 
        noresults1.append(ID)
        noresults2.append(info)
        noresults3.append("No Relevant Results")
        
    elif record['IdList'] != []:
        # this is list of PMIDs from search
        PMID = record['IdList']  
        # input PMID list of articles into efetch, returns XML of articles
        handle = Entrez.efetch(db="pubmed", id=PMID, retmode="xml")
        # parses accordingly
        record2 = Entrez.read(handle)
    
        # run function to get list of dataframes (one dataframe for each person), concat dataframe together
        publications = generator(PMID, record2)
        placeholder.append(publications)
        publications = pd.concat(placeholder)
    else: 
        noresults1.append(ID)
        noresults2.append(info)
        noresults3.append("No Results")
end=time.time()
run_time=end-start
run_time

96.65053176879883

In [14]:
publications

,DOI,Affiliation,Name,Title,Clinical Trials,Date,PeopleID,Journal,Match To,Department,Input Affiliation,Input Name,Mesh
33676426,[10.1186/s12885-021-07922-5],[],[],Osimertinib in advanced EGFR-T790M mutation-po...,NCT03790397,Mar 2021,80,BMC cancer,Oncology|Nutrition Medicine Spain,NaN,RAFAEL LOPEZ CASTRO,NaN,[]
33417511,[10.1200/JCO.20.00550],"[Medical Oncology, Hospital Clínico Universita...",[Rafael López Castro],Nivolumab Plus Ipilimumab for Treatment-Naïve ...,NA,Feb 2021,80,Journal of clinical oncology : official journa...,Oncology|Nutrition Medicine Spain,NaN,RAFAEL LOPEZ CASTRO,NaN,[]
33214227,[10.1136/esmoopen-2020-001021],[],[],Lung cancer symptoms at diagnosis: results of ...,NA,11 2020,80,ESMO open,Oncology|Nutrition Medicine Spain,NaN,RAFAEL LOPEZ CASTRO,NaN,[]
32450544,[10.1016/j.canep.2020.101737],"[Hospital Clínico Universitario de Valladolid,...",[Rafael López Castro],Sex is a strong prognostic factor in stage IV ...,NA,08 2020,80,Cancer epidemiology,Oncology|Nutrition Medicine Spain,NaN,RAFAEL LOPEZ CASTRO,NaN,"[Adult, Aged, Carcinoma, Non-Small-Cell Lung, ..."
30407160,[10.5152/TurkThoracJ.2017.17047],[],[],Forearm Metastases as Unsuspected First Manife...,NA,Nov 2017,80,Turkish thoracic journal,Oncology|Nutrition Medicine Spain,NaN,RAFAEL LOPEZ CASTRO,NaN,[]
27799509,[],[],[],On the Effect of Triplet or Doublet Chemothera...,NA,11 2016,98,Journal of the National Comprehensive Cancer N...,Oncology Spain,NaN,JOSE ENRIQUE LORENZO BARRETO,NaN,"[Adult, Aged, Aged, 80 and over, Antineoplasti..."


In [41]:
#noresults = pd.DataFrame(
#    {'No Results': noresults1,
#     'Search Term': noresults2,
#     'Reason': noresults3
#    })
#pprint.pprint(noresults) 
# print(noresults2)

#noresults.to_excel("Noresults with Missing Name.xlsx")

#now = datetime.now()
#date_time = now.strftime("%Y%m%d")


file_name = "EU5_2"


publications.to_excel(f'{file_name}_Output1.xlsx')
#De empty
Output1=pd.read_excel(f'{file_name}_Output1.xlsx', index_col = None)  
Output1_clear=Output1.drop(Output1[Output1['Name']=='[]'].index)
Output1_clear.to_excel(f'{file_name}_Output2.xlsx')
noresults_2 = pd.DataFrame(
    {'No Results':list(set(list(Output1['PeopleID'][Output1[Output1['Name']=='[]'].index])+noresults1))
    })
pprint.pprint(noresults_2)
noresults_2.to_excel(f'{file_name}Noresults.xlsx')

    No Results
0            2
1            3
2            5
..         ...
91         115
92         116
93         117

[94 rows x 1 columns]


In [42]:
Output1_clear

,Unnamed: 0,DOI,Affiliation,Name,Title,Clinical Trials,Date,PeopleID,Journal,Match To,Department,Input Affiliation,Input Name,Mesh
0,29055469,"[StringElement('10.1016/j.msard.2017.08.015', ...","['Department of Neurology, Galway University H...",['Teresa Leahy'],Clinically isolated syndromes or clinically is...,NaN,Oct 2017,4,Multiple sclerosis and related disorders,Hospital Nurse|Neurology United Kingdom,NaN,Teresa Leahy,NaN,"['Adult', 'Anxiety', 'Attitude of Health Perso..."
1,33837114,"[StringElement('10.1212/WNL.0000000000011998',...","[': Department of Clinical Neurophysiology, Am...",['Alexandre Leclancher'],Cardiac Autonomic Dysfunction and Risk of Sudd...,NaN,Apr 2021,10,Neurology,Neurology France,NaN,ALEXANDRE LECLANCHER,NaN,[]
2,15663612,[],"['Department of Neurology, University of Essen...",['Tobias Leniger'],Comparison of intravenous valproate with intra...,NaN,Jan 2005,27,Headache,Neurology Germany,NaN,Tobias Leniger,NaN,"['Acute Disease', 'Analgesics', 'Aspirin', 'Do..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82,32904925,"[StringElement('10.36185/2532-1900-008', attri...","['Fondazione Policlinico Universitario ""A. Gem...",['Matteo Lucchini'],Estimating the impact of COVID-19 pandemic on ...,NaN,Jun 2020,118,Acta myologica : myopathies and cardiomyopathi...,Neurology Italy,NaN,MATTEO LUCCHINI,NaN,"['Ambulatory Care', 'Betacoronavirus', 'COVID-..."
83,32574249,"[StringElement('10.3389/fneur.2020.00564', att...","[""Dipartimento Scienze dell'Invecchiamento, Ne...",['Matteo Lucchini'],An Italian Neurology Outpatient Clinic Facing ...,NaN,2020,118,Frontiers in neurology,Neurology Italy,NaN,MATTEO LUCCHINI,NaN,[]
84,32506391,"[StringElement('10.1007/s00415-020-09959-1', a...","['Fondazione Policlinico Universitario ""Gemell...",['Matteo Lucchini'],Dimethyl fumarate vs Teriflunomide: an Italian...,NaN,Oct 2020,118,Journal of neurology,Neurology Italy,NaN,MATTEO LUCCHINI,NaN,[]
